# Filter the census data in required format.
The format is in csv and headers like this ->  
'province','district', 'municipality', 'ward', 'gender', 'all_ages', '00-04','05-09','10-14','15-19','20-24','25-29',
                      '30-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70-74','75-79','80-84','85-89','90-94','95+'

In [5]:
# import the required libraries
import pandas as pd
import geopandas as gpd

In [6]:
# Load shape file from folder and see the head
# nepal = gpd.read_file('./shp-local-units/LocalUnit753_Ward_WGS84.shp')
# print(nepal.head(10))
# nepal.plot()
# plt.show()

In [7]:
# Convert excel to csv and txt

census_file = pd.read_excel("census_local_unit_age.xlsx")
census_file.to_csv('census_local_unit_age.txt', index=False, encoding='utf-8')

In [8]:
# Read the converted file
data = pd.read_csv('census_local_unit_age.txt')
print(data.head(100))

   Table 02: Population by five year age groups, NPHC 2021 Unnamed: 1   
0                                                 NaN             NaN  \
1                                               Area              NaN   
2                                                 NaN             NaN   
3                                               KOSHI             NaN   
4                                                 NaN       Taplejung   
..                                                ...             ...   
95                                                NaN             NaN   
96                                                NaN             NaN   
97                                                NaN             NaN   
98                                                NaN             NaN   
99                                                NaN             NaN   

   Unnamed: 2 Unnamed: 3             Unnamed: 4 Unnamed: 5 Unnamed: 6   
0         NaN        NaN                    NaN   

The logic behind the preparing of the data is number of commas(i.e. Column for each data) in each line  
For example if line has 1 comma before any readable string, it has district name  
if line has 2 comma, it has district name.  
Create array for total, male and female and collect the values until another ward is reached, when another ward is reached, copy the values into the output file and clear the array values.  
In this way, data is made in the new format

In [32]:
import re
import csv

# Open converted txt file to process and output_filtered file in which we are going to write into
with open('census_local_unit_age.txt') as file, open('output_filtered.csv', 'w') as out_file:
    # create the row for header and write into the output file
    out_header_row = ['province','district', 'municipality', 'ward', 'gender', 'all_ages', '00-04','05-09','10-14','15-19','20-24','25-29',
                      '30-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70-74','75-79','80-84','85-89','90-94','95+']
    writer = csv.writer(out_file)
    writer.writerow(out_header_row)
    
    
    province = "Koshi"        
    district = ""
    municipality = ""
    ward = ""

    total = []
    male = []
    female = []
    for index, line in enumerate(file):
        # print(index)
        
        if index <= 4:
            continue
        if line[1] != ",":
            district = line.replace(",", "").replace("\n", "")
        if line[2] != "," and line[1] == ",":
            previous_mun = municipality
            municipality = line.replace(",", "").replace("\n", "")
            
        if line[3] != "," and line[2] == ",":   
            temp_ward = line.replace(",","").replace("\n", "")
            out_mun = ""
            if len(total)>0:                
                if "All" in temp_ward:
                    out_mun = previous_mun
                elif "All" not in temp_ward:
                    out_mun = municipality
                    # write to the row                
                total_row = [province, district, out_mun, ward, 'all'] + total
                # print(total_row)
                male_row = [province, district, out_mun, ward, 'male' ] + male
                female_row = [province, district, out_mun, ward, 'female'] + female
                writer.writerow(total_row)
                writer.writerow(male_row)
                writer.writerow(female_row)
                # print(female_row)
                total.clear()
                male.clear()
                female.clear()               
            ward = line.replace(",","").replace("\n", "")
            
        
        if line[4] != "," and line[3] == ",":
            line_temp = line[4:]
            line_data_list = line_temp.split(",")
            # print(line_data_list)
            total.append(line_data_list[1])
            male.append(line_data_list[2])
            female_temp_data = line_data_list[3].replace("\n", "")
            female.append(female_temp_data)
    

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

df = pd.read_csv('output_filtered.csv')   # Read csv
nepal_shp = gpd.read_file('./shp-local-units/LocalUnit753_Ward_WGS84.shp')   # Read shp
merged = nepal_shp.merge(df, on='') # Merge two data according to the key attribute here i.e. dist_name
fig, ax = plt.subplots(figsize=(20, 10)) # Configure the zoom of the plotting
merged.plot(column='Total', cmap='Blues', legend=True, ax=ax) #